# Notebook for Testing the Security System Project

This notebook implements an MQTT client responsible for controlling the QuietWatch project. The goal is to automate the testing procedure and provide a tour of the basic journey of our project.

## Initial Setup

Firstly, it's necessary to install the Paho library to instantiate an MQTT client. Once the library is installed, *this block does not need to be executed again*.

In [1]:
pip install paho-mqtt

Now, let's create variables with the MQTT access credentials. Once the variables are created, *this block does not need to be executed again*.


In [2]:
user = "grupo1-bancadaB5"
passwd = "L@Bdygy1B5"

Now, let's instantiate our MQTT client object. At the time of connection, we will also subscribe to the output topics **S0** (alert_mov) and **S1** (calibrating). Note that the **on_message** function defines our reaction upon receiving messages on these topics. From these reactions, we can verify the results of our tests with the output of our code. *This block does not need to be executed again*.


In [3]:
import paho.mqtt.client as mqtt
import time
from datetime import datetime

broker = "3.141.193.238"       
port = 80            
keepAlive = 60
topico_alerta = user+"/S0"
topico_calibrando = user+"/S1"

def on_connect(client, userdata, flags, rc):
    print("Conectado com codigo " + str(rc))
    client.subscribe(topico_alerta, qos=0)
    client.subscribe(topico_calibrando, qos=0)

def on_message(client, userdata, msg):
    topico = msg.topic
    valor = str(msg.payload)
    if(topico.endswith("0")): #topico S0
      if(valor.endswith("1")): #Alerta de Movimento ativado
        now = datetime.now()
        horario = now.strftime("%H:%M:%S")
        print("ALERTA DE MOVIMENTO " + horario)
    else: #topico S1
      if(valor.endswith("1")): #Calibrando ativado
        now = datetime.now()
        horario = now.strftime("%H:%M:%S")
        print("Calibrando... " + horario)

client = mqtt.Client()                      
client.on_connect = on_connect              
client.on_message = on_message              
client.username_pw_set(user, passwd)        
client.connect(broker, port, keepAlive)

0

# Testing Procedure

After the initial setup, we are ready to publish commands to the input topics **E0** (reset), **E1** (turn on), and **E4** (mode). The QuietWatch project has two operation modes: **at home** and **away from home**. Let's test each one of them.

### Testing "at home" mode

In the "at home" mode, when activating the *alert_mov*, the movement of the servo motor is not interrupted. Since the user is present at home, movement is expected, so the events are simply recorded for later analysis.

In [4]:
client.loop_start()
#primeiro vamos garantir que o valor inicial das entradas eh zero
client.publish(user+"/E0", payload="0", qos=0, retain=False)
client.publish(user+"/E1", payload="0", qos=0, retain=False)
client.publish(user+"/E4", payload="0", qos=0, retain=False) #modo em casa
time.sleep(1)
client.publish(user+"/E0", payload="1", qos=0, retain=False) #operação reset
time.sleep(1)
client.publish(user+"/E0", payload="0", qos=0, retain=False)
client.publish(user+"/E1", payload="1", qos=0, retain=False) #operação ligar
time.sleep(30) #tempo para monitorar resultados
client.loop_stop()

Conectado com codigo 0


At the end of this code block, the circuit is operating in the **at home** mode, and we have 30 seconds to interact with the physical setup while Python monitors the circuit's output topics.

### Testing "away from home" mode

In the "away from home" mode, when activating the *alerta_mov*, the movement of the servo motor is interrupted. The goal is to monitor the position where the movement was detected and only resume normal movement routine after the HC-SR04 sensor measurement returns to normal.

In [3]:
client.loop_start()
#primeiro vamos garantir que o valor inicial das entradas eh zero
client.publish(user+"/E0", payload="0", qos=0, retain=False)
client.publish(user+"/E1", payload="0", qos=0, retain=False)
client.publish(user+"/E4", payload="0", qos=0, retain=False)
time.sleep(1)
client.publish(user+"/E0", payload="1", qos=0, retain=False) #operação reset
client.publish(user+"/E4", payload="1", qos=0, retain=False) #modo fora de casa
time.sleep(1)
client.publish(user+"/E0", payload="0", qos=0, retain=False)
client.publish(user+"/E1", payload="1", qos=0, retain=False) #operação ligar
time.sleep(30) #tempo para monitorar resultados
client.loop_stop()

At the end of this code block, the circuit is operating in the **away from home** mode, and we have 30 seconds to interact with the physical setup while Python monitors the circuit's output topics.